In [1]:
import numpy as np
import pandas as pd
from aif360.datasets import GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing
from IPython.display import Markdown, display

In [2]:
dataset_orig = GermanDataset(
    protected_attribute_names=['age'],                           
    privileged_classes=[lambda x: x >= 25],     
    features_to_drop=['personal_status', 'sex'] 
   )
dataset_orig

               instance weights features                \
                                                         
                                   month credit_amount   
instance names                                           
0                           1.0      6.0        1169.0   
1                           1.0     48.0        5951.0   
2                           1.0     12.0        2096.0   
3                           1.0     42.0        7882.0   
4                           1.0     24.0        4870.0   
...                         ...      ...           ...   
995                         1.0     12.0        1736.0   
996                         1.0     30.0        3857.0   
997                         1.0     12.0         804.0   
998                         1.0     45.0        1845.0   
999                         1.0     45.0        4576.0   

                                                                \
                                                               

In [3]:
dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

In [4]:
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

In [5]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups for age before handling the bias is %f" % metric_orig_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups for age before handling the bias is -0.066667


In [6]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_orig_train)  

In [7]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups in age after handling the bias is %f" % metric_transf_train.mean_difference())

#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups in age after handling the bias is 0.000000


## Part 2
Mitigating Bias in Sex column

In [8]:
dataset_orig = GermanDataset(
    protected_attribute_names=['sex'],                           
    privileged_classes=[lambda x: x == 'male'],     
    features_to_drop=['personal_status', 'age'] 
   )
dataset_orig

               instance weights features                \
                                                         
                                   month credit_amount   
instance names                                           
0                           1.0      6.0        1169.0   
1                           1.0     48.0        5951.0   
2                           1.0     12.0        2096.0   
3                           1.0     42.0        7882.0   
4                           1.0     24.0        4870.0   
...                         ...      ...           ...   
995                         1.0     12.0        1736.0   
996                         1.0     30.0        3857.0   
997                         1.0     12.0         804.0   
998                         1.0     45.0        1845.0   
999                         1.0     45.0        4576.0   

                                                                \
                                                               

In [9]:
dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

In [10]:
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

In [11]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups for sex before handling the bias is %f" % metric_orig_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups for sex before handling the bias is -0.062147


In [12]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_orig_train)

In [13]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups for sex after handling the bias is %f" % metric_transf_train.mean_difference())

#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups for sex after handling the bias is 0.000000


The major advantage of Machine Learning models over statistical models is that the Machine Learning models are very good at identifying and learning patterns in the given data in order to make predictions. But, as much as selective learning is an advantage of Machine Learning, it is also its major disadvantage. The Machine learning models are trained to predict the outcomes based on a certain property and the if the past data that is suppied to train the model has bias in it, then the model introduces a bias in all the results by default. When we consider the example of Admission to an academic institute, if the past records of the institute indicate that less women have been gievn admission, then the model filters out female candidates by default. Such problems may arise at any step of the model pipeline and it is very importnat to handle these biases if we want to create a fairly unbiased model which does not supress some results while boosting some others.

In the tutorial that is implemeted above in the notebook, we can see that bias can be introduced due to any feature in the dataset.It is very important to handle bias if we want to ensure that the model is fair and not biased towards some feature categories and supresses all other feature categories. In the tutrial, the guide shows a way in which one can check for the bias in the dataset and the possible wrong classification of results that is possible. The steps for the guide are fairly simple. We have to import the dataset that has to be tested, give the protected variables and the other categorical variables which are supposed to be dropped. We then split the data into training and testing dataset. After that we assign previlege groups to the class labels, in order to check for the difference in mean ratings of the groups. If the value is very small or close to 0, we can assume that the feature does not introduce much bias in the model. However, if the value is larger, then we need to downweigh the privileged and unprivileged groups and consequently check if the score has improved. If the score still does not improve, then we should consider recollecting the data and/or dropping the feature from the predictive model.